1. Seleccionar datos relevantes

Customers: customer_id, customer_city, customer_state.

Orders: order_id, customer_id, order_status, order_purchase_timestamp, order_delivered_timestamp, order_estimated_delivery_date.

OrderItems: order_id, product_id, seller_id, price, shipping_charges.

In [1]:
import pandas as pd
data_path = "../data/01_raw/"

# Cargar datasets
customers = pd.read_csv(data_path + 'df_Customers.csv')
orders = pd.read_csv(data_path + 'df_Orders.csv')
order_items = pd.read_csv(data_path + 'df_OrderItems.csv')

# Dimensiones
print("Customers:", customers.shape)
print("Orders:", orders.shape)
print("Order Items:", order_items.shape)

# Primeras filas
customers.head()
orders.head()
order_items.head()

Customers: (89316, 4)
Orders: (89316, 7)
Order Items: (89316, 5)


,order_id,product_id,seller_id,price,shipping_charges
0,Axfy13Hk4PIk,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65
1,v6px92oS8cLG,qejhpMGGVcsl,IjlpYfhUbRQs,170.80,23.79
2,Ulpf9skrhjfm,qUS5d2pEAyxJ,77p2EYxcM9MD,64.40,17.38
3,bwJVWupf2keN,639iGvMyv0De,jWzS0ayv9TGf,264.50,30.72
4,Dd0QnrMk9Cj5,1lycYGcsic2F,l1pYW6GBnPMr,779.90,30.66


Limpieza de datos

In [2]:
import numpy as np
# Eliminar duplicados
customers_clean = customers.drop_duplicates()
orders_clean = orders.drop_duplicates()
order_items_clean = order_items.drop_duplicates()

# Estandarizar texto en ciudad y estado
customers_clean["customer_city"] = customers_clean["customer_city"].str.lower().str.strip()
customers_clean["customer_state"] = customers_clean["customer_state"].str.upper().str.strip()

# Rellenar valores nulos en ciudad/estado con 'desconocido'
customers_clean["customer_city"] = customers_clean["customer_city"].fillna("desconocido")
customers_clean["customer_state"] = customers_clean["customer_state"].fillna("DESCONOCIDO")

# Ejemplo: eliminar pedidos sin fecha de entrega
orders_clean = orders_clean.dropna(subset=["order_delivered_timestamp", "order_estimated_delivery_date"])

# Eliminar pedidos cancelados
orders_clean = orders_clean[orders_clean["order_status"] != "canceled"]

# Outliers en precios y envíos (corte al percentil 99)
price_cap = order_items_clean["price"].quantile(0.99)
shipping_cap = order_items_clean["shipping_charges"].quantile(0.99)

order_items_clean["price"] = np.where(order_items_clean["price"] > price_cap, price_cap, order_items_clean["price"])
order_items_clean["shipping_charges"] = np.where(order_items_clean["shipping_charges"] > shipping_cap, shipping_cap, order_items_clean["shipping_charges"])

# Mostrar resumen de nulos
print("Nulos en customers:\n", customers_clean.isnull().sum())
print("Nulos en orders:\n", orders_clean.isnull().sum())
print("Nulos en order_items:\n", order_items_clean.isnull().sum())

Nulos en customers:
 customer_id                 0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
Nulos en orders:
 order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                9
order_delivered_timestamp        0
order_estimated_delivery_date    0
dtype: int64
Nulos en order_items:
 order_id            0
product_id          0
seller_id           0
price               0
shipping_charges    0
dtype: int64


Feature Engineering (nuevas variables)

Retraso en la entrega: (delivered - estimated).days.

Tiempo real de entrega: (delivered - purchase).days.

Ticket total por pedido: price + shipping_charges.

Año/Mes de compra: para análisis temporal.

In [3]:
# Convertir fechas a tipo datetime
orders_clean["order_purchase_timestamp"] = pd.to_datetime(orders_clean["order_purchase_timestamp"])
orders_clean["order_delivered_timestamp"] = pd.to_datetime(orders_clean["order_delivered_timestamp"])
orders_clean["order_estimated_delivery_date"] = pd.to_datetime(orders_clean["order_estimated_delivery_date"])

# Variables derivadas
orders_clean["delivery_delay"] = (orders_clean["order_delivered_timestamp"] - orders_clean["order_estimated_delivery_date"]).dt.days
orders_clean["delivery_time"] = (orders_clean["order_delivered_timestamp"] - orders_clean["order_purchase_timestamp"]).dt.days
orders_clean["order_year_month"] = orders_clean["order_purchase_timestamp"].dt.to_period("M")

# Ticket total por pedido
order_items_clean["total_item_value"] = order_items_clean["price"] + order_items_clean["shipping_charges"]

# Ejemplo de agregación: ticket promedio por pedido
ticket_por_pedido = order_items_clean.groupby("order_id")["total_item_value"].sum().reset_index()
ticket_por_pedido.rename(columns={"total_item_value": "order_total_value"}, inplace=True)

# Unir ticket total al dataframe de pedidos
orders_clean = orders_clean.merge(ticket_por_pedido, on="order_id", how="left")

# Mostrar primeras filas del resultado final
display(orders_clean.head())

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,delivery_delay,delivery_time,order_year_month,order_total_value
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,-14,4,2017-10,308.16
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,-21,13,2018-06,194.59
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,-9,10,2018-02,81.78
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,-23,9,2018-08,295.22
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,-13,14,2017-12,810.56


Integración de múltiples fuentes

In [4]:
# Seleccionar columnas relevantes de customers
customers_sel = customers_clean[["customer_id", "customer_city", "customer_state"]]

# Merge de Orders con Customers
orders_customers = orders_clean.merge(customers_sel, on="customer_id", how="inner")

# Merge con OrderItems
df_final = orders_customers.merge(order_items_clean, on="order_id", how="inner")

# Dimensiones del dataset final
print("Dimensiones del dataset final:", df_final.shape)

# Mostrar primeras filas del dataset final
display(df_final.head())

Dimensiones del dataset final: (87422, 18)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,delivery_delay,delivery_time,order_year_month,order_total_value,customer_city,customer_state,product_id,seller_id,price,shipping_charges,total_item_value
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,-14,4,2017-10,308.16,varzea paulista,SP,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65,308.16
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24,-21,13,2018-06,194.59,armacao dos buzios,RJ,qejhpMGGVcsl,IjlpYfhUbRQs,170.80,23.79,194.59
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08,-9,10,2018-02,81.78,jandira,SP,qUS5d2pEAyxJ,77p2EYxcM9MD,64.40,17.38,81.78
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19,-23,9,2018-08,295.22,uberlandia,MG,639iGvMyv0De,jWzS0ayv9TGf,264.50,30.72,295.22
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18,-13,14,2017-12,810.56,ilhabela,SP,1lycYGcsic2F,l1pYW6GBnPMr,779.90,30.66,810.56


Transformación y formateo para ML

Codificación categórica: customer_state, customer_city, order_status.

Normalización/escalado de variables numéricas.

Generar dataset final model_input listo para entrenar.

In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Codificación de variables categóricas
encoder = LabelEncoder()
df_final["customer_state_enc"] = encoder.fit_transform(df_final["customer_state"])
df_final["order_status_enc"] = encoder.fit_transform(df_final["order_status"])

# Escalado de variables numéricas
scaler = StandardScaler()
df_final[["price_scaled", "shipping_scaled", "delivery_time_scaled"]] = scaler.fit_transform(
    df_final[["price", "shipping_charges", "delivery_time"]]
)

# Selección final de features para ML
model_input = df_final[[
    "customer_state_enc", "order_status_enc", 
    "price_scaled", "shipping_scaled", "delivery_time_scaled", "delivery_delay"
]]

# Mostrar primeras filas del dataset final
display(model_input.head())

,customer_state_enc,order_status_enc,price_scaled,shipping_scaled,delivery_time_scaled,delivery_delay
0,25,0,-0.215422,1.182372,-0.858948,-14
1,18,0,-0.316818,-0.573275,0.111146,-21
2,25,0,-0.521495,-0.758186,-0.212219,-9
3,10,0,-0.136571,-0.373363,-0.320007,-23
4,25,0,0.854882,-0.375094,0.218934,-13


Guardar datasets en carpetas de Kedro

In [6]:
# Guardar datasets limpios y finales
customers_sel.to_csv("../data/02_intermediate/customers_clean.csv", index=False)
orders_clean.to_csv("../data/02_intermediate/orders_clean.csv", index=False)
order_items_clean.to_csv("../data/02_intermediate/order_items_clean.csv", index=False)
df_final.to_csv("../data/03_primary/full_dataset.csv", index=False)
model_input.to_csv("../data/05_model_input/model_input.csv", index=False)